### imports

In [1]:
import numpy as np
import strange
import toytree
import ipyparallel as ipp

### make tree

In [2]:
rtr = toytree.rtree().coaltree(ntips=8,seed=42)
rtr3 = rtr.mod.node_scale_root_height(3)
rtr3.draw();

r7 r2 r1 r4 r3 r0 r5 r6

### simulate!

In [3]:
Ne = 50000
mut = 1e-8
kwargs = {
    "workdir": "../tests",
    "mutation_rate": mut,
    "recombination_rate": 1e-9,
    "theta": Ne*mut*4,
    "length": int(1e6), 
    "get_sequences": True,
    "random_seed": 42,
}

# simulation object
coal8 = strange.Coalseq(tree=rtr3, name="coal8", **kwargs)

In [4]:
coal8.tree_table.head(10)

,end,length,mstree,nsnps,start,treeheight
0,54,54,"((8:204293.32434399396880,(6:64174.00872287491...",4,0,473009
1,568,514,"((8:204293.32434399396880,(6:64174.00872287491...",6,54,473009
2,675,107,"((8:204293.32434399396880,(6:64174.00872287491...",2,568,473009
3,2454,1779,"((8:204293.32434399396880,(6:64174.00872287491...",31,675,473009
4,2515,61,"((8:204293.32434399396880,(6:64174.00872287491...",1,2454,473009
5,2655,140,"((8:204293.32434399396880,(6:64174.00872287491...",1,2515,546145
6,2960,305,"((8:204293.32434399396880,(6:64174.00872287491...",5,2655,546145
7,3420,460,"((8:204293.32434399396880,(6:64174.00872287491...",8,2960,546145
8,3690,270,"(((4:104748.95632937064511,5:104748.9563293706...",8,3420,546145
9,4288,598,"(((4:104748.95632937064511,5:104748.9563293706...",8,3690,546145


### Let's run MrBayes on each gene tree

In [5]:
ipyclient = ipp.Client()
ipyclient

In [6]:
sliding_obj = strange.SlidingWindow(name='coal8',workdir='../tests/',ipyclient=ipyclient)

In [7]:
sliding_obj.run_mb_mstrees()

[####################] 100% 0:14:43 | inferring mb trees on mstrees 
consolidating...
done.


### So now we've run MrBayes on each gene tree, made a key that has indexed every observed topology, and produced a table of how often each topology was visited during each gene tree mcmc. 

We might want to also compute the probability, for each visited topology, of that topology being produced under the species tree. We can do that!

In [8]:
sliding_obj.add_probs_topokey()

[####################] 100% 0:03:51 | computing gene tree probabilities 

### Now let's get closer to mcmc world:

make an mcmc object:

In [9]:
mcmc_obj = strange.MBmcmc(name = 'coal8',workdir = '../tests/')

### each column in this array corresponds to an msprime gene segment that has been been MrBayesed, and each row corresponds to a topology visited by MrBayes.

In [10]:
mcmc_obj.mbcsv

array([[9062, 1969, 1008, ..., 6972,  555,  155],
       [9062, 1969, 1008, ..., 6972,  555,  155],
       [9062, 1969, 1008, ..., 6972,  555,  155],
       ...,
       [ 264,  590, 6206, ..., 2559, 8985, 3238],
       [ 264,  590, 6206, ..., 2559, 8985, 3238],
       [ 264,  590, 6206, ..., 2559, 8985, 3238]])

### we can also score this array (rmse -- so we want to minimize this value) based on how closely the "mode" topology for each column matches its expected gene tree frequency:

In [11]:
mcmc_obj.score()

0.007375994919759486

### Now, we can resample a bunch of times:

In [12]:
mcmc_obj.update_x_times(mixnum=5, # the number of entries replaced in each column for each round
                      num_times = 500, # the number of rounds
                      sd_normal = 1) # the sd of the normal distribution used to determine
                                     # which column to drawn from (independent normal draws for each col)

### after 500 resamples...

In [13]:
mcmc_obj.mbcsv

array([[9062, 8985, 7668, ..., 6972,  555, 3812],
       [9657, 6583, 2949, ..., 6972,  555,  155],
       [9062, 6149, 2122, ..., 6972,  555,  155],
       ...,
       [4764, 1969, 6206, ..., 6972, 8985, 3238],
       [9657,  590, 6206, ..., 3239, 8985, 3238],
       [ 264,  590, 6206, ..., 6919,  555,  155]])

### ...you can see that it looks a bit different.

### has the score changed?

In [14]:
mcmc_obj.score()

0.0072169621300013425

### yep, slightly.

###  

## Room for speed improvement:

### Right now, the scoring function is particularly slow:

In [15]:
%%timeit
mcmc_obj.score()

1 loop, best of 3: 238 ms per loop


### i.e. we're only able to score ~15,000 times per hour.

### The resampling isn't too much better:

In [16]:
%%timeit
mcmc_obj.update_x_times(mixnum=5, 
                      num_times = 500,
                      sd_normal = 1)

1 loop, best of 3: 9.88 s per loop


### almost 10 seconds per 500 resamplings.